# Using a fourier transform and a deep CNN to classify SHL timeseries data

In [1]:
import pandas as pd

location = 'shl-dataset/train/Hand/'

acc_x = pd.read_csv(location + 'Acc_x.txt', header=None, sep=' ').to_numpy()
print('Acc_x Import Done')
acc_y = pd.read_csv(location + 'Acc_y.txt', header=None, sep=' ').to_numpy()
print('Acc_y Import Done')
acc_z = pd.read_csv(location + 'Acc_z.txt', header=None, sep=' ').to_numpy()
print('Acc_z Import Done')

labels = pd.read_csv(location + 'Label.txt', header=None, sep=' ').mode(axis=1).to_numpy().flatten()
print('Label Import Done')

Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Label Import Done


In [2]:
import numpy as np

# Compute magnitudes of ACC, MAG and GYR
acc_mag = np.sqrt(acc_x**2 + acc_y**2 + acc_z**2)

In [3]:
from sklearn.model_selection import train_test_split

X = acc_mag
y = np.eye(9)[labels] # OHE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)

# Shuffle train set
idx = np.random.permutation(len(X_train))
X_train = X_train[idx]
y_train = y_train[idx]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(176464, 500) (176464, 9) (19608, 500) (19608, 9)


In [4]:
import tensorflow as tf
from tensorflow import keras

# Check that we can use our GPU, to not wait forever during training
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12187321235553849658,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3215029044
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13391783513135154025
 physical_device_desc: "device: 0, name: GeForce GTX 770, pci bus id: 0000:01:00.0, compute capability: 3.0"]

In [13]:
nb_classes = 9
n_feature_maps = 64

model = keras.models.Sequential([
    keras.layers.Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
    keras.layers.Lambda(lambda v: tf.cast(tf.spectral.fft(tf.cast(v,dtype=tf.complex64)),tf.float32)),

    # Block 1
    keras.layers.Conv1D(n_feature_maps, kernel_size=8, activation='relu', input_shape=(X_train.shape[1], 1)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(n_feature_maps, kernel_size=5, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(n_feature_maps, kernel_size=3, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),

    # Block 2
    keras.layers.Conv1D(n_feature_maps * 2, kernel_size=8, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(n_feature_maps * 2, kernel_size=5, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(n_feature_maps * 2, kernel_size=3, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),

    # Block 3
    keras.layers.Conv1D(n_feature_maps * 2, kernel_size=8, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(n_feature_maps * 2, kernel_size=5, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(n_feature_maps * 2, kernel_size=3, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),

    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(nb_classes, activation='softmax')
])

# categorical_hinge

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
)

In [14]:
# Model training

epochs = 20
batch_size = 32

callbacks = [
    keras.callbacks.TensorBoard(log_dir='./log'),
    keras.callbacks.ModelCheckpoint(
        'shl-model.stft.h5', save_best_only=True, monitor='val_loss'
    ),
]

history = model.fit(
    X_train, 
    y_train,
    epochs=epochs, 
    batch_size=batch_size, 
    callbacks=callbacks,
    validation_split=0.1,
    verbose=1,
)

Train on 158817 samples, validate on 17647 samples
Epoch 1/20
158817/158817 [==============================] - 469s 3ms/step - loss: 1.0270 - acc: 0.5990 - val_loss: 1.4092 - val_acc: 0.5128
Epoch 2/20
158817/158817 [==============================] - 465s 3ms/step - loss: 0.8951 - acc: 0.6531 - val_loss: 2.1026 - val_acc: 0.4398
Epoch 3/20
  1664/158817 [..............................] - ETA: 7:36 - loss: 0.9103 - acc: 0.6454

KeyboardInterrupt: 